In [ ]:
!pip install torch torchvision torchaudio
!pip install wandb
!pip install kaggle
!pip install tqdm matplotlib seaborn


In [ ]:
from google.colab import files
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip -d data/


In [ ]:
import wandb
wandb.login()


In [ ]:
import wandb
from src.data_loader import FERDataset
from src.train import train_model
from torch.utils.data import DataLoader
from torchvision import transforms

def run_pipeline(model_class, architecture_name, config_dict):
    wandb.init(project="facial_expression_recognition", config=config_dict)
    wandb.config["architecture"] = architecture_name


    transform = transforms.Compose([
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_dataset = FERDataset(csv_file='data/train_split.csv', transform=transform)
    val_dataset = FERDataset(csv_file='data/val_split.csv', transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=wandb.config.batch_size, shuffle=False)


    model = model_class()


    train_model(model, train_loader, val_loader, wandb.config)

    wandb.finish()


In [ ]:
from src.model import SimpleCNN, DeeperCNN


import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/train.csv')

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['emotion'])

train_df.to_csv('data/train_split.csv', index=False)
val_df.to_csv('data/val_split.csv', index=False)

print("Train/val split done.")



run_pipeline(SimpleCNN, "SimpleCNN", {
    "epochs": 15,
    "learning_rate": 1e-3,
    "batch_size": 64
})


run_pipeline(DeeperCNN, "DeeperCNN", {
    "epochs": 25,
    "learning_rate": 1e-3,
    "batch_size": 64
})


In [ ]:
from src.model import ResNet18


run_pipeline(ResNet18, "ResNet18", {
    "epochs": 30,
    "learning_rate": 1e-4,
    "batch_size": 64
})


In [ ]:
from src.model import DeeperCNN2

run_pipeline(DeeperCNN2, "DeeperCNN2", {
    "epochs": 25,
    "learning_rate": 1e-3,
    "batch_size": 64
})
